In [41]:
import os
import json
import pytz
import pandas as pd
import numpy as np
from pathlib import Path
from ast import literal_eval
# import Source.db_functions as db

from dateutil.tz import tz
from sqlalchemy import engine

## Main functions to manipulate a dataframe

### For testing purposes of data manipulation functions

In [2]:
def get_files_in_directory() -> list:
    """
    Sets a path to JSON file.
    :returns a file name in a set path
    """
    path_to_files = '../data/input/'
    files_in_path = os.scandir(path_to_files)
    
    list_of_files = []
    for file in files_in_path:
        if file.is_dir() or file.is_file():
            list_of_files.append(file.name)
            return list_of_files    # <- don't forget to align this statement back with for!!!!!

In [3]:
def create_dataframe(file_json: str) -> pd.DataFrame:
    """
    Creates a pandas dataframe from JSON file.
    Sets the maximum available columns to be shown.
    Requires name of the file.
    """
    path_to_files = '../data/input/'
    with open(path_to_files + file_json) as jfile:
        json_data = json.load(jfile)
        df = pd.DataFrame(pd.json_normalize(json_data))
        pd.set_option('display.max_columns', None)
    return df

In [4]:
# FLATTEN JSON FILE!!!
def flatten_json_file(dataframe: pd.DataFrame, row: str) -> pd.DataFrame:
    """
    Flattens the supplied dataframe and returns a new dataframe with flattened json data.
    :param dataframe: dataframe to flatten
    :return: new dataframe with flattened json data
    """
    dataframe_flat = dataframe[row].apply(pd.Series)
    dataframe_flat_0 = dataframe_flat[0].apply(pd.Series)
    dataframe_flat_0.columns = ['weather_id', 'weather_main', 'weather_description', 'weather_icon']
    dataframe_new = pd.concat([dataframe, dataframe_flat_0], axis=1)
    dataframe_new = dataframe_new.drop(columns=['weather'], axis=1)
    return dataframe_new

In [5]:
def change_column_names(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Changes the column names of the dataframe to their new column names.
    :param dataframe: a pandas dataframe to change column names
    :return: dataframe with new column names
    """
    new_names = {"dt": "date_local",
                 "name": "city",
                 "id": "country_id", 
                 "coord.lon": "longitude",
                 "coord.lat": "latitude",
                 "main.temp": "main_temp",
                 "main.feels_like": "main_feels_like",
                 "main.temp_min": "main_temp_min",
                 "main.temp_max": "main_temp_max",
                 "main.pressure": "pressure",
                 "main.humidity": "humidity",
                 "wind.speed": "wind_speed",
                 "wind.deg": "wind_deg",
                 "clouds.all": "clouds",
                 "sys.type": "sys_type",
                 "sys.id": "sys_id",
                 "sys.country": "country",
                 "sys.sunrise": "sunrise",
                 "sys.sunset": "sunset"}
    dataframe.rename(columns=new_names, inplace=True)
    return dataframe

In [42]:
def change_datetime_format(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Change datetime format of a given dataframe to ISO 8601 format
    :param dataframe: dataframe to change datetime format for.
    :return: dataframe with changed datetime format
    """
    date_cols = ['date_local', 'sunrise', 'sunset']
    local_timezone = pytz.timezone('Europe/Vilnius')

    for i in date_cols:
        dataframe[i] = (pd.to_datetime(dataframe[i], unit='s', errors='coerce', utc=True)
                        )
    dataframe['date_vilnius'] = dataframe['date_local'].dt.tz_convert(local_timezone)
    return dataframe

In [7]:
def reorder_dataframe_columns(dataframe: pd.DataFrame) -> pd.DataFrame:
    reordered_columns = ['country_id', 'country', 'city', 'longitude', 'latitude', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max', 'date_vilnius', 'date_local', 'timezone', 'sunrise', 'sunset', 'weather_id', 'weather_main', 'weather_description', 'weather_icon', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds', 'visibility', 'base', 'sys_type', 'sys_id', 'cod']
    
    dataframe = dataframe[reordered_columns]
    return dataframe

In [8]:
def inspect_dataframe(dataframe: pd.DataFrame):
    """
    Returns information about the dataset 
    and shows a preview of the data in it.
    """
    print(dataframe.info(), '\n')
    print(dataframe.head())

In [9]:
def load_to_database(dataframe: pd.DataFrame, table_name: str) -> None:
    """
    Function to load the data of a dataframe to a specified table in the database.
    :param dataframe: dataframe to load data from.
    :param table_name: table to load the data to.
    :return: None
    """
    
    dataframe.to_sql(table_name, engine, if_exists='append')

## Finding the longest value in each column of a given dataframe

In [10]:
def longest_value(dataframe: pd.DataFrame):
    """
    Checking for the longest value in each column of a given dataframe.
    """
    for row in dataframe:
        col_name = row

        if dataframe[row].dtype == float or int:
            list_len = dataframe[row].astype(str).str.len().max()

        else:
            list_len = max(list(map(len, dataframe[row].values)))

        print('Column:', col_name)
        print('Value length:', list_len, '\n')

## Files in ../data/input location

In [50]:
cities = get_files_in_directory()
print(cities)

['Barcelona, Spain_response.json']


In [55]:
for city in cities:
    city_df = create_dataframe(city)

city_df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1708012741,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,1707979649,1708017828


In [56]:
flatten_df = flatten_json_file(city_df, 'weather')
flatten_df.head()

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,weather_id,weather_main,weather_description,weather_icon
0,stations,10000,1708012741,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,1707979649,1708017828,800,Clear,clear sky,01d


In [57]:
names_df = change_column_names(flatten_df)
names_df.head()

,base,visibility,date,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sys_type,sys_id,country,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon
0,stations,10000,1708012741,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,1707979649,1708017828,800,Clear,clear sky,01d


In [58]:
# normalize date to my local timezone
import pytz

local_timezone = pytz.timezone('Europe/Vilnius')
names_df['date'] = pd.to_datetime(names_df['date'], unit='s', errors='coerce', utc=True)
names_df['date_df'] = names_df['date'].dt.tz_convert(local_timezone)

# names_df['date_local'] = names_df['date_df'].dt.tz_convert(local_timezone)
print(local_timezone)
names_df.head()

Europe/Vilnius


,base,visibility,date,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sys_type,sys_id,country,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon,date_df
0,stations,10000,2024-02-15 15:59:01+00:00,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,1707979649,1708017828,800,Clear,clear sky,01d,2024-02-15 17:59:01+02:00


In [60]:
dates_df = change_datetime_format(flatten_df)
dates_df.head()

,base,visibility,date,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sys_type,sys_id,country,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon,date_df
0,stations,10000,2024-02-15 17:59:01+02:00,3600,3128760,Barcelona,200,2.159,41.3888,16.9,16.47,14.71,20.72,1017,70,3.09,110,0,2,18549,ES,2024-02-15 08:47:29+02:00,2024-02-15 19:23:48+02:00,800,Clear,clear sky,01d,2024-02-15 17:59:01+02:00


In [69]:
dates_df.head()

,base,visibility,date,timezone,country_id,city,cod,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sys_type,sys_id,country,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon
0,stations,10000,2024-02-07 11:01:05,10800,745044,Istanbul,200,28.9497,41.0138,14.04,13.38,14.04,17.45,1015,72,2.06,230,0,1,6970,TR,2024-02-07 05:09:28,2024-02-07 15:27:10,800,Clear,clear sky,01d


In [86]:
columns = dates_df.columns.tolist()

index_col = []
for i, col in enumerate(columns):
    index_col.append([i, col])
    
print(index_col)

[[0, 'base'], [1, 'visibility'], [2, 'date'], [3, 'timezone'], [4, 'country_id'], [5, 'city'], [6, 'cod'], [7, 'longitude'], [8, 'latitude'], [9, 'main_temp'], [10, 'main_feels_like'], [11, 'main_temp_min'], [12, 'main_temp_max'], [13, 'pressure'], [14, 'humidity'], [15, 'wind_speed'], [16, 'wind_deg'], [17, 'clouds'], [18, 'sys_type'], [19, 'sys_id'], [20, 'country'], [21, 'sunrise'], [22, 'sunset'], [23, 'weather_id'], [24, 'weather_main'], [25, 'weather_description'], [26, 'weather_icon']]


In [61]:
reordered_columns = ['country_id', 'country', 'city', 'longitude', 'latitude', 'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max', 'date', 'timezone', 'sunrise', 'sunset', 'weather_id', 'weather_main', 'weather_description', 'weather_icon', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds', 'visibility', 'base', 'sys_type', 'sys_id', 'cod']

reordered_df = dates_df[reordered_columns]

reordered_df.head()

,country_id,country,city,longitude,latitude,main_temp,main_feels_like,main_temp_min,main_temp_max,date,timezone,sunrise,sunset,weather_id,weather_main,weather_description,weather_icon,pressure,humidity,wind_speed,wind_deg,clouds,visibility,base,sys_type,sys_id,cod
0,3128760,ES,Barcelona,2.159,41.3888,16.9,16.47,14.71,20.72,2024-02-15 17:59:01+02:00,3600,2024-02-15 08:47:29+02:00,2024-02-15 19:23:48+02:00,800,Clear,clear sky,01d,1017,70,3.09,110,0,10000,stations,2,18549,200


In [109]:
city_df_val_len = longest_value(reordered_df)

Column: country_id
Value length: 6 

Column: country
Value length: 2 

Column: city
Value length: 8 

Column: longitude
Value length: 7 

Column: latitude
Value length: 7 

Column: main_temp
Value length: 5 

Column: main_feels_like
Value length: 5 

Column: main_temp_min
Value length: 5 

Column: main_temp_max
Value length: 5 

Column: date
Value length: 19 

Column: timezone
Value length: 5 

Column: sunrise
Value length: 19 

Column: sunset
Value length: 19 

Column: weather_id
Value length: 3 

Column: weather_main
Value length: 5 

Column: weather_description
Value length: 9 

Column: weather_icon
Value length: 3 

Column: pressure
Value length: 4 

Column: humidity
Value length: 2 

Column: wind_speed
Value length: 4 

Column: wind_deg
Value length: 3 

Column: clouds
Value length: 1 

Column: visibility
Value length: 5 

Column: base
Value length: 8 

Column: sys_type
Value length: 1 

Column: sys_id
Value length: 4 

Column: cod
Value length: 3 


In [110]:
city_inspect = inspect_dataframe(reordered_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   country_id           1 non-null      int64         
 1   country              1 non-null      object        
 2   city                 1 non-null      object        
 3   longitude            1 non-null      float64       
 4   latitude             1 non-null      float64       
 5   main_temp            1 non-null      float64       
 6   main_feels_like      1 non-null      float64       
 7   main_temp_min        1 non-null      float64       
 8   main_temp_max        1 non-null      float64       
 9   date                 1 non-null      datetime64[ns]
 10  timezone             1 non-null      int64         
 11  sunrise              1 non-null      datetime64[ns]
 12  sunset               1 non-null      datetime64[ns]
 13  weather_id           1 non-null      in